In [7]:
# Merge transactions with product details including 'Category' column
df = pd.merge(transactions, products[['ProductID', 'Price', 'Category']], on='ProductID', how='left')

# Check the column names to confirm 'Category' is included
print(df.columns)

# Use 'Price_x' if it exists, otherwise 'Price_y'
df['Price'] = df['Price_x'].fillna(df['Price_y'])

# Step 1: Aggregate customer features
customer_features = df.groupby("CustomerID").agg(
    Total_Spending=("TotalValue", "sum"),
    Total_Quantity=("Quantity", "sum"),
    Avg_Price=("Price", "mean"),
    Favorite_Category=("Category", lambda x: x.mode()[0] if not x.mode().empty else "Unknown")
).reset_index()

# Step 2: Scale the customer features
scaler = StandardScaler()
customer_scaled = scaler.fit_transform(customer_features[["Total_Spending", "Total_Quantity", "Avg_Price"]])  # Scale the relevant columns

# Step 3: Apply KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)
customer_features["Cluster"] = kmeans.fit_predict(customer_scaled)

# Step 4: Compute Davies-Bouldin Index for cluster evaluation
db_index = davies_bouldin_score(customer_scaled, customer_features["Cluster"])

# Print the results
print(f"Davies-Bouldin Index: {db_index}")


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'Price_y', 'Category'],
      dtype='object')
Davies-Bouldin Index: 1.1218019122669298


C:\Users\amaan\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
